## LionForests Multi Label Experiments: FoodTrucks Dataset

In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from datasets.dataset import Dataset
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import warnings
import random
import numpy as np
import pandas as pd
from utilities.dummy_utilizer import DummyUtilizer
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from algorithms.MARLENA.marlena.marlena.marlena import MARLENA
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.CHIRPS.structures import data_container
import algorithms.CHIRPS.routines as rt
import algorithms.CHIRPS.structures as strcts
from scipy import sparse
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

np.seterr(invalid='ignore')
warnings.filterwarnings("ignore")

Load Dataset

In [3]:
foodtrucks = Dataset()
X_p, y, feature_names, label_names = foodtrucks.load_foodtrucks()

Initialize LionForests

In [4]:
from lionforestsmulti import LionForests
lf = LionForests(None, False, None, list(feature_names), label_names, categorical_features=['x0','x1'])#,'x2']

In [5]:
parameters = [{
    'max_depth': [10],
    'max_features': [0.75],
    'bootstrap': [True],
    'min_samples_leaf' : [1],
    'n_estimators': [1000]
}]
lf.fit(X_p, y, parameters, True)

Best RF model

In [6]:
lf.model

RandomForestClassifier(max_depth=10, max_features=0.75, n_estimators=1000,
                       n_jobs=-1, random_state=0)

Example explanation for option "all predicted labelsets"

In [7]:
lf.explain_n_wise(X_p[2],'all',True, method='1')[0]

'if 0.5<=frequency numeric<=1.0 & 15.0<=expenses numeric<=17.5 & 4.5<=taste numeric<=5.0 & 1.5<=hygiene numeric<=2.5 & 1.5<=menu numeric<=2.0 & 4.5<=presentation numeric<=5.0 & 2.5<=attendance numeric<=3.5 & 4.5<=ingredients numeric<=5.0 & 1.5<=place.to.sit numeric<=2.5 & 1.5<=takeaway numeric<=2.0 & 1.5<=variation numeric<=2.0 & 1.5<=stop.strucks numeric<=2.0 & 1.0<=schedule numeric<=1.5 & 1.5<=age.group numeric<=2.5 & 1.25<=scholarity numeric<=1.75 & 3.5<=average.income numeric<=4.5 & 0.5<=has.work numeric<=1.0 & x0_by_chance & x1_afternoon & 1.5<=gender<=2.0 & 2.5<=marital.status<=3.0 then street_food brazilian_food'

Example explanation for option "per label" for label "street food"

In [8]:
lf.explain_n_wise(X_p[2],'per label',True, method='1')['street_food'][0]

'if 0.5<=frequency numeric<=1.0 & 15.0<=expenses numeric<=17.5 & 4.5<=taste numeric<=5.0 & 1.5<=hygiene numeric<=2.5 & 1.5<=menu numeric<=2.5 & 4.5<=presentation numeric<=5.0 & 2.5<=attendance numeric<=3.5 & 4.5<=ingredients numeric<=5.0 & 1.5<=place.to.sit numeric<=2.5 & 1.5<=takeaway numeric<=2.0 & 1.5<=variation numeric<=2.0 & 1.5<=stop.strucks numeric<=2.5 & 1.0<=schedule numeric<=1.5 & 1.5<=age.group numeric<=2.5 & 1.25<=scholarity numeric<=1.75 & 3.5<=average.income numeric<=4.5 & 0.5<=has.work numeric<=1.0 & x0_by_chance & x1_afternoon & 1.5<=gender<=2.0 & 2.5<=marital.status<=3.0 then street_food'

Example explanation for option "frequent pairs"

In [9]:
lf.explain_n_wise(X_p[2],'frequent pairs',7, True, method='1')[('street_food',)][0]

'if 0.5<=frequency numeric<=1.0 & 15.0<=expenses numeric<=17.5 & 4.5<=taste numeric<=5.0 & 1.5<=hygiene numeric<=2.5 & 1.5<=menu numeric<=2.5 & 4.5<=presentation numeric<=5.0 & 2.5<=attendance numeric<=3.5 & 4.5<=ingredients numeric<=5.0 & 1.5<=place.to.sit numeric<=2.5 & 1.5<=takeaway numeric<=2.0 & 1.5<=variation numeric<=2.0 & 1.5<=stop.strucks numeric<=2.5 & 1.0<=schedule numeric<=1.5 & 1.5<=age.group numeric<=2.5 & 1.25<=scholarity numeric<=1.75 & 3.5<=average.income numeric<=4.5 & 0.5<=has.work numeric<=1.0 & x0_by_chance & x1_afternoon & 1.5<=gender<=2.0 & 2.5<=marital.status<=3.0 then street_food'

## Experiments

1. Comparison between LF variations:

In [ ]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [1000]
    }]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    lf = LionForests(None, False, scaler, feature_names,
                     label_names, categorical_features=['x0', 'x1'])
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[8]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][8].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        def lf_rule_pairs(instance):
            temp = lf.explain_n_wise(
                instance, 'frequent pairs', len(label_names))
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][8].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        return {'lf-a': lf_rule_all, 'lf-l': lf_rule_per_label, 'lf-p': lf_rule_pairs}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, label_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    rule_length = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    f_precision = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    time_response = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    rules = {'lf-a': [], 'lf-l': [], 'lf-p': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0
    for tesd_ind in range(len(test)):
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'lf-p':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        labeles = []
                        for kk in key:
                            labeles.append(list(label_names).index(kk))
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                temp_prediction = [y_test_temp[tesd_ind][j] for j in range(
                                    len(y_test_temp[tesd_ind])) if j in labeles]
                                temperatura = [y_test_temp[co][j] for j in range(
                                    len(y_test_temp[co])) if j in labeles]
                                precision.append(
                                    [temp_prediction, temperatura])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, 0], precision[:, 1], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10)
folds = 0
test_size = []
for train_index, test_index in kf.split(X_p):
    X_train, X_test = X_p[train_index], X_p[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds=folds+1

Results

In [ ]:
full_coverage = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
rule_length = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_precision ={'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_time = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

2. Comparison between LF-a and multi-label explainability techniques


In [ ]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [1000]
    }]
    lf = LionForests(None, False, None, feature_names,
                     label_names, ['x0', 'x1'])
    lf.fit(X_train, y_train, params=parameters)
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[8]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        gt = GlobalSurrogateTree(
            train, predictions, feature_names, task, random_state)
        print('    GT Ready')
        lt = LocalSurrogateTree(
            train, predictions, feature_names, task, 150, random_state)

        def marlena(instance):
            m1 = MARLENA(neigh_type='mixed', random_state=42)
            i2e = pd.Series(instance, index=feature_names)
            X2E = pd.DataFrame(train, columns=feature_names)
            #num_feature_names = num_features + ordinal_categorical_features
            #categorical_fs = list(preprocess.named_transformers_['one-hot'].named_steps['one_hot'].get_feature_names())
            rule, _, _, _, _ = m1.extract_explanation(i2e, X2E, lf.model, feature_names, [],  # categorical_fs,
                                                      label_names, k=10, size=50, alpha=0.7)

            path = rule.split('->')[0][1:-2]
            prediction = rule.split('->')[1][1:]
            conjuctions = {}
            print(path)
            if '{}' in rule:
                print(rule)
            else:
                for conj in path.split('\n'):
                    temp_conj = conj.strip(',')
                    if temp_conj[0] == ' ':
                        temp_conj = temp_conj[1:]
                    if '>' in temp_conj:
                        temp = temp_conj.split(' > ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '>':
                                    position = 1
                            if conjuctions[temp[0]][position][0] == '>':
                                if conjuctions[temp[0]][position][1] > float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '<=':
                                conjuctions[temp[0]].append(
                                    ['>', float(temp[1])])

                        else:
                            conjuctions[temp[0]] = [['>', float(temp[1])]]
                    else:
                        temp = temp_conj.split(' <= ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '<=':
                                    position = 1

                            if conjuctions[temp[0]][position][0] == '<=':
                                if conjuctions[temp[0]][position][1] <= float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '>':
                                conjuctions[temp[0]].append(
                                    ['<=', float(temp[1])])
                        else:
                            conjuctions[temp[0]] = [['<=', float(temp[1])]]
            local_prediction = np.zeros((len(label_names)), dtype=int)
            for label in range(len(label_names)):
                if label_names[label] in prediction:
                    local_prediction[label] = 1
            return (conjuctions, local_prediction)

        return {'lf-a': lf_rule_all, 'gs': gt.rule, 'ls': lt.rule, 'ma': marlena}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, label_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    rule_length = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    f_precision = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    time_response = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    rules = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'ls' or name == 'gs' or name == 'ma':
                    ts = time.time()
                    rule, prediction = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp:
                        res = rule_cov(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(prediction), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp)
                    rules[name].append(rule)
                    len_rule = len(rule)
                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state = 77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X_p):
    X_train, X_test = X_p[train_index], X_p[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds = folds + 1

Results

In [ ]:
full_coverage = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
rule_length = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_precision ={'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_time = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

3. Comparison between LF-l and "per label" explainability techniques (CHIRPS/ANCHORS)

In [26]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [1000]
    }]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    lf = LionForests(None, False, scaler, feature_names, label_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules
        inddd = 0
        return {'lf-l': lf_rule_per_label}

        chts = time.time()
        chirps = {}
        inddd = 0
        for label_name in label_names:
            project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
            temp_frame = pd.DataFrame(np.hstack((train, y_train[:, inddd].reshape(
                len(y_train), 1))), columns=feature_names+['class'])
            temp_frame['class'] = temp_frame['class'].astype(int)

            mydata = data_container(
                data=temp_frame, class_col='class', var_names=feature_names,
                project_dir=project_dir, save_dir='FT'+str(inddd), random_state=random_state)
            meta_data = mydata.get_meta()
            f_walker = strcts.classification_trees_walker(
                forest=model, inddd=inddd, meta_data=meta_data)
            f_walker.forest_walk(instances=test, labels=model.predict(test)[
                                 :, inddd], forest_walk_async=False)

            explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                                   forest=model,
                                                   # any representative sample can be used
                                                   sample_instances=sparse.csr_matrix(
                                                       train),
                                                   sample_labels=predictions[:, inddd],
                                                   meta_data=meta_data)
            explanations.run_explanations(target_classes=model.predict(test)[:, inddd],  # we're explaining the prediction, not the true label!
                                          explanation_async=False,
                                          random_state=random_state,
                                          which_trees='majority',
                                          alpha_paths=0.0,
                                          support_paths=0.1,
                                          score_func=1,
                                          precis_threshold=0.8,
                                          disc_path_bins=2,
                                          merging_bootstraps=10,
                                          pruning_bootstraps=10,
                                          delta=0.2)
            chirps[inddd] = explanations
            inddd += 1
        chte = (time.time() - chts)/len(test)

        def chirps_method(instance, idx):
            prediction = lf.model.predict([instance])[0]
            rules = {}
            for pred in range(len(prediction)):
                if prediction[pred] == 1:
                    chirps_dict = {}
                    explanations = chirps[pred]
                    for i in explanations.explainers[idx].pruned_rule:
                        if i[1]:
                            chirps_dict[i[0]] = [['<=', float(i[2])]]
                        else:
                            chirps_dict[i[0]] = [['>', float(i[2])]]
                    rules[label_names[pred]] = chirps_dict
            return rules, 0, chte

        return {'lf-l': lf_rule_per_label}

        return {'lf-l': lf_rule_per_label, 'an': anchors_method, 'ch': chirps_method}

    interpretation = techniques(lf.model, train, y_train, predictions,
                                test, feature_names, label_names, lf, 'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-l': 0, 'an': 0, 'ch': 0}
    rule_length = {'lf-l': 0, 'an': 0, 'ch': 0}
    f_precision = {'lf-l': [], 'an': [], 'ch': []}
    time_response = {'lf-l': [], 'an': [], 'ch': []}
    rules = {'lf-l': [], 'an': [], 'ch': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)

    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:

            for name, method in rule_generator.items():
                if name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'an':
                    ts = time.time()
                    rule = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'ch':
                    rule, op, te = method(x_test_temp[tesd_ind], tesd_ind)
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return _, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame


In [27]:
feature_names2 = [i.replace(' ','_') for i in feature_names]

In [ ]:
#total_results = []
kf = KFold(n_splits=10, random_state=77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X_p):
    if True:
        X_train, X_test = X_p[train_index], X_p[test_index]
        y_train, y_test = y[train_index], y[test_index]
        test_size.append(len(X_test))
        print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
        results = measure(X_train, X_test, y_train, y_test, feature_names2, label_names)
        total_results.append(results)
    folds=folds+1

Results

In [32]:
full_coverage = {'lf-l':[], 'an':[], 'ch':[]}
rule_length = {'lf-l':[], 'an':[], 'ch':[]}
f_precision = {'lf-l':[], 'an':[], 'ch':[]}
f_time = {'lf-l':[], 'an':[], 'ch':[]}
k = 0
for i in total_results[10:]:
    for name in ['lf-l']:
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name in ['lf-l']:
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

lf-l | 0.0245  0.001 | 41.4805 4.763 | 1.0000  0.000 | 10.2610  0.895


In [21]:
full_coverage = {'lf-l':[], 'an':[], 'ch':[]}
rule_length = {'lf-l':[], 'an':[], 'ch':[]}
f_precision = {'lf-l':[], 'an':[], 'ch':[]}
f_time = {'lf-l':[], 'an':[], 'ch':[]}
k = 0
for i in total_results:
    for name in ['ch']:
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name in ['ch']:
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

ch | 0.7285  0.088 | 3.6631 1.114 | 0.8990  0.028 | 8.4652  0.353


In [18]:
#ANCHORS:

an | 0.0642  0.016 | 8.9537 2.301 | 0.9859  0.017 | 341.6183  105.925
